<a href="https://colab.research.google.com/github/Mubassir1820/Kaggle_competition/blob/main/Kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6,y
0,0,A 0,B0,C2,D1,100,E1,F2,237000
1,1,A1,B0,C11,D4,100,E4,F2,86193
2,2,A0,B0,C18,D0,0,E0,F2,169200
3,3,A2,B0,C11,D1,100,E1,F2,58000
4,4,A0,B0,C67,D1,0,E1,F2,235000


In [4]:
df.shape

(2628, 9)

In [5]:
df['col_0'].value_counts()

A0        1517
A1         489
A2         178
A3          67
 A0         53
A 0         38
AO          36
A0          35
A 1         21
A1          16
 A1         16
 A2          7
A 2          6
A2           4
 A3          4
A 3          1
A   3        1
Name: col_0, dtype: int64

In [6]:
df['col_1'].value_counts()

B0        2375
 B0         48
B 0         33
B0          33
BO          29
B3          10
B2           7
B1           5
B   3        1
B1           1
Name: col_1, dtype: int64

In [7]:
df['col_2'].value_counts()

C11    728
C2     587
C4     416
C9     204
C15     77
      ... 
C62      1
C63      1
C92      1
C76      1
C66      1
Name: col_2, Length: 90, dtype: int64

In [8]:
df['col_3'].value_counts()

D1     2093
D4      116
D0       61
D2       60
D6       54
       ... 
D32       1
D71       1
D75       1
D66       1
D73       1
Name: col_3, Length: 68, dtype: int64

In [9]:
df['col_5'].value_counts()

E1     2120
E4      121
E2       62
E0       57
E6       44
       ... 
E45       1
E47       1
E20       1
E67       1
E62       1
Name: col_5, Length: 67, dtype: int64

In [10]:
df['col_6'].value_counts()

F2        1976
F0         286
F1          97
F 2         46
F2          43
 F2         39
F0           7
F 0          6
 F0          6
FO           4
F 1          3
F1           2
 F1          1
Name: col_6, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628 entries, 0 to 2627
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UID     2628 non-null   int64 
 1   col_0   2489 non-null   object
 2   col_1   2542 non-null   object
 3   col_2   2628 non-null   object
 4   col_3   2628 non-null   object
 5   col_4   2628 non-null   int64 
 6   col_5   2628 non-null   object
 7   col_6   2516 non-null   object
 8   y       2628 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 184.9+ KB


In [12]:
df.describe()

,UID,col_4,y
count,2628.000000,2628.000000,2628.000000
mean,1313.500000,46.308980,137642.657154
std,758.782578,48.558188,62886.926575
min,0.000000,0.000000,5132.000000
25%,656.750000,0.000000,95000.000000
50%,1313.500000,0.000000,135000.000000
75%,1970.250000,100.000000,176325.000000
max,2627.000000,100.000000,423000.000000


In [13]:
df.value_counts()

UID   col_0   col_1  col_2  col_3  col_4  col_5  col_6  y     
0     A 0     B0     C2     D1     100    E1     F2     237000    1
1752  A0      B0     C18    D2     0      E2     F2     248100    1
1745  A0      B0     C2     D1     100    E1     F2     185000    1
1746  A1      B0     C11    D1     100    E1     F2     60400     1
1747   A0     B0     C20    D1     100    E1     F0     100000    1
                                                                 ..
864   A0      B 0    C39    D1     100    E1     F1     190000    1
863   A0      B0     C11    D1     100    E1     F2     220110    1
862   A0      B0     C11    D1     100    E1     F2     160000    1
861   A0      B0     C11    D1     0      E1     F2     63000     1
2627  A0      B0     C2     D1     100    E1     F2     191475    1
Length: 2301, dtype: int64

In [14]:
df.isnull().sum()


UID        0
col_0    139
col_1     86
col_2      0
col_3      0
col_4      0
col_5      0
col_6    112
y          0
dtype: int64

In [15]:
# Replace the missing values with mode
df['col_0'].fillna(df['col_0'].mode()[0], inplace=True)
df['col_1'].fillna(df['col_1'].mode()[0], inplace=True)
df['col_6'].fillna(df['col_6'].mode()[0], inplace=True)

In [16]:
df.isnull().sum()

UID      0
col_0    0
col_1    0
col_2    0
col_3    0
col_4    0
col_5    0
col_6    0
y        0
dtype: int64

In [17]:
# Regular expression to clean the data
import re

def preprocess(text):
  text = re.sub('[a-z^\s]','',text)
  return text

In [18]:
df['col_0'] = df['col_0'].map(preprocess)

In [19]:
df['col_0'].value_counts()

A0    1782
A1     542
A2     195
A3      73
AO      36
Name: col_0, dtype: int64

In [20]:
df['col_1'] = df['col_1'].map(preprocess)

In [21]:
df['col_1'].value_counts()

B0    2575
BO      29
B3      11
B2       7
B1       6
Name: col_1, dtype: int64

In [22]:
df['col_6'] = df['col_6'].map(preprocess)

In [23]:
df['col_6'].value_counts()

F2    2216
F0     305
F1     103
FO       4
Name: col_6, dtype: int64

In [24]:
df.head(10)

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6,y
0,0,A0,B0,C2,D1,100,E1,F2,237000
1,1,A1,B0,C11,D4,100,E4,F2,86193
2,2,A0,B0,C18,D0,0,E0,F2,169200
3,3,A2,B0,C11,D1,100,E1,F2,58000
4,4,A0,B0,C67,D1,0,E1,F2,235000
5,5,A0,B0,C11,D1,100,E1,F2,170000
6,6,AO,B0,C11,D1,100,E1,F0,183600
7,7,A0,B0,C4,D1,0,E1,F2,100000
8,8,A0,B0,C2,D1,100,E1,F2,175000
9,9,A0,B0,C2,D1,100,E1,F2,136000


In [25]:
### Encoding of the categorical values

In [26]:
#one hot encoding
df = pd.get_dummies(df, columns = ['col_0','col_1','col_6'])

In [27]:
df.head()

,UID,col_2,col_3,col_4,col_5,y,col_0_A0,col_0_A1,col_0_A2,col_0_A3,col_0_AO,col_1_B0,col_1_B1,col_1_B2,col_1_B3,col_1_BO,col_6_F0,col_6_F1,col_6_F2,col_6_FO
0,0,C2,D1,100,E1,237000,1,0,0,0,0,1,0,0,0,0,0,0,1,0
1,1,C11,D4,100,E4,86193,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,2,C18,D0,0,E0,169200,1,0,0,0,0,1,0,0,0,0,0,0,1,0
3,3,C11,D1,100,E1,58000,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,4,C67,D1,0,E1,235000,1,0,0,0,0,1,0,0,0,0,0,0,1,0


In [28]:
# Count encoding of column 2, 3 , 5
count_encoding1 = df['col_2'].value_counts().to_dict()
df['col_2_count_encoded'] = df['col_2'].map(count_encoding1)

count_encoding2 = df['col_3'].value_counts().to_dict()
df['col_3_count_encoded'] = df['col_3'].map(count_encoding2)

count_encoding3 = df['col_5'].value_counts().to_dict()
df['col_5_count_encoded'] = df['col_5'].map(count_encoding3)

In [29]:
df.head()

,UID,col_2,col_3,col_4,col_5,y,col_0_A0,col_0_A1,col_0_A2,col_0_A3,...,col_1_B2,col_1_B3,col_1_BO,col_6_F0,col_6_F1,col_6_F2,col_6_FO,col_2_count_encoded,col_3_count_encoded,col_5_count_encoded
0,0,C2,D1,100,E1,237000,1,0,0,0,...,0,0,0,0,0,1,0,587,2093,2120
1,1,C11,D4,100,E4,86193,0,1,0,0,...,0,0,0,0,0,1,0,728,116,121
2,2,C18,D0,0,E0,169200,1,0,0,0,...,0,0,0,0,0,1,0,59,61,57
3,3,C11,D1,100,E1,58000,0,0,1,0,...,0,0,0,0,0,1,0,728,2093,2120
4,4,C67,D1,0,E1,235000,1,0,0,0,...,0,0,0,0,0,1,0,7,2093,2120


In [30]:
# Drop the uid column
columns_to_drop = ['UID', 'y', 'col_2', 'col_3', 'col_5']
X = df.drop(columns = columns_to_drop)

In [31]:
X.head()

,col_4,col_0_A0,col_0_A1,col_0_A2,col_0_A3,col_0_AO,col_1_B0,col_1_B1,col_1_B2,col_1_B3,col_1_BO,col_6_F0,col_6_F1,col_6_F2,col_6_FO,col_2_count_encoded,col_3_count_encoded,col_5_count_encoded
0,100,1,0,0,0,0,1,0,0,0,0,0,0,1,0,587,2093,2120
1,100,0,1,0,0,0,1,0,0,0,0,0,0,1,0,728,116,121
2,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,59,61,57
3,100,0,0,1,0,0,1,0,0,0,0,0,0,1,0,728,2093,2120
4,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,7,2093,2120


In [32]:
y = df['y'].values

In [33]:
# Dataset splitting

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [34]:
import xgboost as xgb
from xgboost import XGBRegressor
model = XGBRegressor(
    n_estimators=200,   # Number of trees (boosting rounds)
    learning_rate=0.1,  # Step size shrinkage
    max_depth=3,        # Maximum depth of each tree
    random_state=42     # Seed for reproducibility
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [35]:
#make prediction
y_pred = model.predict(X_test)

In [36]:
#Evaluation
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Mean Squared Error: 2337581870.966111
Root Mean Squared Error: 48348.54569649548
R-squared: 0.41626586579843927


In [37]:
#Hyperparameter tuning and cross validation
from sklearn.model_selection import GridSearchCV

# Define a parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

In [38]:
print(best_params)

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [39]:
# Prediction on test data
test_data = pd.read_csv('test.csv')
test_data.head()

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6
0,2628,A0,B0,C11,D1,100,E1,F2
1,2629,A0,B0,C11,D1,0,E1,F2
2,2630,A0,B0,C11,D1,100,E1,F2
3,2631,A0,B0,C26,D1,100,E1,F0
4,2632,A0,B0,C11,D1,100,E1,NaN


In [41]:
test_data.isnull().sum()

UID       0
col_0    67
col_1    34
col_2     0
col_3     0
col_4     0
col_5     0
col_6    48
dtype: int64

In [42]:
# Replace the missing values with mode
test_data['col_0'].fillna(test_data['col_0'].mode()[0], inplace=True)
test_data['col_1'].fillna(test_data['col_1'].mode()[0], inplace=True)
test_data['col_6'].fillna(test_data['col_6'].mode()[0], inplace=True)

In [43]:
test_data.isnull().sum()

UID      0
col_0    0
col_1    0
col_2    0
col_3    0
col_4    0
col_5    0
col_6    0
dtype: int64

In [46]:
test_data.shape, df.shape

((1127, 8), (2628, 23))

In [47]:
test_data['col_0'].value_counts()

A0        710
A1        202
A2         97
A3         31
A 0        18
 A0        17
AO         14
A0         14
 A1         5
A 1         5
A1          4
A 2         3
A2          2
A 3         2
 A2         2
A   3       1
Name: col_0, dtype: int64

In [48]:
test_data['col_1'].value_counts()

B0        1055
B 0         17
BO          15
B0          15
 B0         13
B3           6
B1           4
B2           2
Name: col_1, dtype: int64

In [49]:
test_data['col_2'].value_counts()

C11    312
C2     253
C4     196
C9      85
C7      27
C15     24
C18     23
C26     20
C3      14
C6      11
C50     10
C1       9
C19      9
C27      9
C34      8
C36      7
C20      5
C12      5
C47      5
C33      4
C25      4
C49      4
C42      4
C53      4
C38      4
C29      4
C84      3
C69      3
C87      3
C66      3
C45      3
C57      3
C39      3
C46      3
C65      3
C78      3
C54      3
C30      3
C72      3
C0       2
C85      2
C52      2
C13      2
C28      2
C73      2
C21      2
C56      2
C17      2
C44      1
C63      1
C41      1
C5       1
C32      1
C59      1
C70      1
C76      1
C35      1
C8       1
Name: col_2, dtype: int64

In [50]:
test_data['col_3'].value_counts()

D1     911
D4      51
D2      25
D0      19
D6      17
D12     15
D3      14
D8       8
D37      6
D41      5
D13      5
D9       4
D33      4
D26      3
D16      3
D64      3
D19      2
D25      2
D39      2
D57      2
D24      1
D17      1
D5       1
D11      1
D23      1
D42      1
D20      1
D61      1
D10      1
D22      1
D45      1
D63      1
D18      1
D29      1
D52      1
D62      1
D74      1
D55      1
D15      1
D48      1
D69      1
D43      1
D46      1
D44      1
D67      1
D70      1
Name: col_3, dtype: int64

In [51]:
test_data['col_4'].value_counts()

0      580
100    494
50      53
Name: col_4, dtype: int64

In [52]:
test_data['col_5'].value_counts()

E1     920
E4      51
E2      25
E0      20
E3      18
E6      14
E11     13
E24      7
E33      6
E19      6
E14      3
E23      3
E64      3
E8       3
E44      2
E35      2
E42      2
E22      2
E49      2
E37      2
E9       2
E5       1
E25      1
E21      1
E10      1
E38      1
E17      1
E58      1
E20      1
E13      1
E15      1
E16      1
E60      1
E68      1
E55      1
E53      1
E48      1
E65      1
E39      1
E43      1
E51      1
E66      1
Name: col_5, dtype: int64

In [53]:
test_data['col_6'].value_counts()

F2        914
F0        123
F1         36
F2         16
 F2        14
F 2        13
 F0         7
F 0         3
F 1         1
Name: col_6, dtype: int64

In [56]:
# preprocess with the function preprocess
test_data['col_0'] = test_data['col_0'].map(preprocess)

In [57]:
test_data['col_0'].value_counts()

A0    759
A1    216
A2    104
A3     34
AO     14
Name: col_0, dtype: int64

In [58]:
test_data['col_1'] = test_data['col_1'].map(preprocess)

In [59]:
test_data['col_1'].value_counts()

B0    1100
BO      15
B3       6
B1       4
B2       2
Name: col_1, dtype: int64

In [60]:
test_data['col_6'] = test_data['col_6'].map(preprocess)

In [61]:
test_data['col_6'].value_counts()

F2    957
F0    133
F1     37
Name: col_6, dtype: int64